# Feladat: SAT-solverrel keressünk r db n*n-es, páronként ortogonális latin négyzetet

## Változók:

x_{i,j,k,l} = indikátora annak, hogy az i. sor j. oszlopában a k. szimbólum van az l. latin négyzetben

i, j, k \in {0,...,n-1}

l \in {1,...,r}

In [10]:
#imports
import pysat
import numpy as np
import matplotlib as plt
import pandas as pd
from pysat.solvers import Glucose3
from pysat.card import *

In [11]:
#változók generálása
n=6
r=2
valtozok=dict()
x=1
for i in range(n):
    for j in range(n):
        for k in range(n):
            for l in range(r):
                valtozok[tuple([i, j, k, l])] = x
                x=x+1
#print(valtozok)

In [12]:
ssolver = Glucose3()
normalforma=CNF()

In [13]:
# Feltételek: 
# 1.: minden latin négyzet minden cellájában pontosan 1 szimbólum szerepel

maxvaltozo = r*n**3
for l in range(r):
    for i in range(n):
        for j in range(n):
            l1=[]
            for k in range(n):
                l1=l1+[valtozok[(i,j, k, l)]]
        
            cnf=CardEnc.equals(lits=l1, bound=1, top_id=maxvaltozo)
            for cl in cnf.clauses:
                normalforma.append(cl)
    
            #az abselemek lista tartalmazza az eddig használt változók abszolút értékét
            #a top_id frissítéséhez használom a maxvaltozo-t
            abselemek=[]
            for kloz in normalforma.clauses:
                l2=[abs(elem) for elem in kloz]
                abselemek=abselemek+l2
            maxvaltozo=max(abselemek)

In [14]:
# 2.: Minden szimbólumból minden latin négyzet minden sorában pontosan egy van

for l in range(r):
    for i in range(n):
        for k in range(n):
            l1=[]
            for j in range(n):
                l1=l1+[valtozok[(i,j, k, l)]]
        
            cnf=CardEnc.equals(lits=l1, bound=1, top_id=maxvaltozo)
            for cl in cnf.clauses:
                normalforma.append(cl)
    
            #az abselemek lista tartalmazza az eddig használt változók abszolút értékét
            #a top_id frissítéséhez használom a maxvaltozo-t
            abselemek=[]
            for kloz in normalforma.clauses:
                l2=[abs(elem) for elem in kloz]
                abselemek=abselemek+l2
            maxvaltozo=max(abselemek)

In [15]:
# 3.: Minden szimbólumból minden latin négyzet minden oszlopában pontosan egy elem van

for l in range(r):
    for j in range(n):
        for k in range(n):
            l1=[]
            for i in range(n):
                l1=l1+[valtozok[(i,j, k, l)]]
        
            cnf=CardEnc.equals(lits=l1, bound=1, top_id=maxvaltozo)
            for cl in cnf.clauses:
                normalforma.append(cl)
    
            #az abselemek lista tartalmazza az eddig használt változók abszolút értékét
            #a top_id frissítéséhez használom a maxvaltozo-t
            abselemek=[]
            for kloz in normalforma.clauses:
                l2=[abs(elem) for elem in kloz]
                abselemek=abselemek+l2
            maxvaltozo=max(abselemek)

In [16]:
# 4.: minden latin négyzetben minden szimbólumból pontosan n db van

for l in range(r):
    for k in range(n):
        l1=[]
        for i in range(n):
            for j in range(n):
                l1=l1+[valtozok[(i, j, k, l)]]
                
        cnf=CardEnc.equals(lits=l1, bound=n, top_id=maxvaltozo)
        for cl in cnf.clauses:
            normalforma.append(cl)

        #az abselemek lista tartalmazza az eddig használt változók abszolút értékét
        #a top_id frissítéséhez használom a maxvaltozo-t
        abselemek=[]
        for kloz in normalforma.clauses:
            l2=[abs(elem) for elem in kloz]
            abselemek=abselemek+l2
        maxvaltozo=max(abselemek)

In [17]:
# A latin négyzetek páronként legyenek ortogonálisak:
# Ne legyen két latin négyzet, amiben van 2 cella, amikben szereplő szimbólumpárok megegyeznek.

for l1 in range(r):
    for l2 in range(r):
        for i1 in range(n):
            for j1 in range(n):
                for i2 in range(n):
                    for j2 in range(n):
                        for k1 in range(n):
                            for k2 in range(n):
                                if l1 != l2 and i1 != i2 and j1 != j2:
                                    x1 = valtozok[(i1, j1, k1, l1)]
                                    x2 = valtozok[(i1, j1, k2, l2)]
                                    x3 = valtozok[(i2, j2, k1, l1)]
                                    x4 = valtozok[(i2, j2, k2, l2)]
                                    normalforma.append([-x1, -x2, -x3, -x4])
                            

In [ ]:
#beadom a sat-solvernek a normálformát
for kloz in normalforma.clauses:
    ssolver.add_clause(kloz)
    
# megoldás kiszámítása
ssolver.solve()
ans=ssolver.get_model()
print(ans)